# 摘要
1. 從 reference 資料夾，載入 填值表。
2. 根據 填值表，從 Nas_data 中讀取指定圖表。
3. 根據 填值表，指定圖表之算式，將變數表中的變數，經過算式運算以後，填入 Nas_data 之中。
4. 根據 填值表，將欄位名稱全部改名。
5. 輸出 Nas_data，至此，公開資料的自動化處理結束。預期輸出正確之檔案。

In [ ]:
import pandas, numpy, os, traceback, re

In [ ]:
# 載入填質表
reference_excel_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "reference"), "填值表.xlsx")
reference_excel = pandas.read_excel(reference_excel_path)
reference_excel.head(3)

In [ ]:
Nas_data_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "reference"), "Step1_data.xlsx")
Nas_data = pandas.read_excel(Nas_data_path)
Nas_data[Nas_data["學校代碼"] == "0001"]



In [ ]:
# 載入可先串接資料庫
data_base_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "reference"), "Step3_data_base_concat_frist.xlsx")
data_base = pandas.read_excel(data_base_path)
data_base = data_base[data_base["學年度"].notnull()]
data_base["學年度"] = data_base["學年度"].astype(int) # 將學年度轉為字串型別。以配合後續的填值。
data_base["學年度"] = data_base["學年度"].astype(str)
data_base[data_base["學校代碼"] == "0001"]


# 創建 output 資料夾

In [ ]:
error_Folder_path = os.path.join(os.path.dirname(os.getcwd()), "output")
if not os.path.exists(error_Folder_path):
    os.makedirs(error_Folder_path)
else :
    pass

# 填值

## 使用先串接之資料庫

In [ ]:
for picture_name in list(reference_excel["使用圖型"]) : 

    # 處理可先串接之圖型
    if reference_excel[reference_excel["使用圖型"] == picture_name]["能否先串接"].values[0]  == "Yes" : 
        try : 
            Nas_picture_path = os.path.join(os.path.join(os.path.dirname(os.getcwd()), "output"), f"{picture_name}.xlsx")
            with pandas.ExcelWriter(Nas_picture_path, engine="openpyxl") as writer:
                
                # 使用表冊、(學)年度、公式工作表_建立
                reference_excel_copy = reference_excel[reference_excel["使用圖型"] == picture_name][["使用資料來源", "公式", "資料下載時間"]].copy()
            
                # 將資料下載時間，轉換為時間型別
                reference_excel_copy["資料下載時間"] = reference_excel_copy["資料下載時間"].dt.date
                
                reference_excel_copy.to_excel(writer, sheet_name = "使用表冊、(學)年度、公式", index = False)

                # 最終結果工作表_建立。
                Nas_picture_data = pandas.read_excel(Nas_data_path, sheet_name= picture_name)
                
                # 為每一張圖表，新增一個新的學年度欄位。如，原本只有 110 年，則多一欄 111年。
                Nas_picture_data[str(int(Nas_picture_data.columns[-1]) + 1)] = 0

                # 載入該圖表對應的公式。備註，這個是字串型別的公式，要轉譯才能使用。
                formula = reference_excel[reference_excel["使用圖型"] == picture_name]["公式(pandas)"].values[0]
                
                # 將公式字串做轉換，data["???"]，轉為，sch_open_data["???"]
                formula = formula.replace("data", "sch_year_data")
                formula = formula.replace("]", "].values[0]")
                # print(formula)

                try : 
                    # 根據公式，將公開資料庫的數據，填入Nas_data之中。並根據能否先串接，選用不同的資料庫。
                    for sch_index in list(dict.fromkeys(Nas_picture_data["學校代碼"])) : 
                        sch_data_base = data_base[data_base["學校代碼"] == sch_index]

                        for sch_year in list(dict.fromkeys(sch_data_base["學年度"])) : 
                            sch_year_data = sch_data_base[sch_data_base["學年度"] == sch_year]
                            # 填入值
                            input_number = round(eval(formula), 9)

                            try : # 將 sch_open_data 中，存在的學年度，盡可能填入 Nas_picture_data 之中。# eval() 轉譯用的函式。

                                if str(int(sch_year)) in Nas_picture_data.columns : # 僅寫入 Nas_picture_data 中，有的學年度資料。
                                    if reference_excel[reference_excel["使用圖型"] == picture_name]["上限為100"].values[0] == "Yes" : # 選出使用百分比單位的圖表。# 註冊率超過100%者，以100%計算。
                                        if input_number > 100 :
                                            input_number = 100
                                            Nas_picture_data.loc[Nas_picture_data["學校代碼"] == sch_index, str(int(sch_year))] = input_number # 遺漏值不論加什麼都會輸出遺漏值，所以"NA"必然被遺漏值取代。
                                        else : 
                                            Nas_picture_data.loc[Nas_picture_data["學校代碼"] == sch_index, str(int(sch_year))] = input_number
                                    else : 
                                        Nas_picture_data.loc[Nas_picture_data["學校代碼"] == sch_index, str(int(sch_year))] = input_number
                            
                        
                            except Exception as e : 
                                error_message = traceback.format_exc()
                                with open(os.path.join(os.path.dirname(os.getcwd()), "error", "可以先串接資料庫(填值_Error)"), "w", encoding='utf-8') as f:
                                    f.write(error_message)
                                                         
                except Exception as e : 
                    error_message = traceback.format_exc()
                    with open(os.path.join(os.path.dirname(os.getcwd()), "error", "公式計算錯誤"), "w", encoding='utf-8') as f:
                        f.write(error_message)

                # 更改欄位名稱，加上填值表的變數名稱。
                column_name = reference_excel[reference_excel["使用圖型"] == picture_name]["變數名稱"].values[0]
                # 每一個分頁的第五行開始，皆是數值變數。
                sch_year_column_list = Nas_picture_data.columns[4:]

                # 欄位改名之結果 list。
                rename_list = [column_name + "_" + str(sch_year_column) for sch_year_column in sch_year_column_list]

                rename_dict = dict(zip(sch_year_column_list, rename_list))
                # print(rename_dict)

                Nas_picture_data = Nas_picture_data.rename(columns = rename_dict)

                # 將所有的遺漏值，以NA字串取代。
                Nas_picture_data = Nas_picture_data.fillna("NA")

                # 將無窮大數值，以NA取代。
                Nas_picture_data.replace(numpy.inf, "NA")

                # 最終結果工作表_儲存
                Nas_picture_data.to_excel(writer, sheet_name = "最終結果", index=False)


                # 來源資料分頁_建立，存放 每一學年度的計算數據，來源數值。(#分流還沒做)
                for sch_year in list(data_base["學年度"]) : # 這裡選哪一個資料都沒差，都有全部的學年度。
                    if f"{column_name}_{str(int(sch_year))}" in Nas_picture_data.columns : # 僅寫入 Nas_picture_data 中，有的學年度資料。

                        sch_year_data = data_base[data_base["學年度"] == sch_year].copy()

                        # 透過正則表達式，抓出本圖表會用到的變數，並組成的 list。
                        var_list = re.findall(re.compile(r'\["(.*?)"\]'), formula)

                        # 分組變數，所組成的 list。如 學校代碼、學年度 等
                        classfication_list = ["學校代碼", "學年度"]

                        sch_year_data = sch_year_data[(classfication_list + var_list)]

                        # 遺漏值以 "NA" 字串輸出
                        sch_year_data = sch_year_data.fillna("NA")
                        sch_str = int(sch_year)
                        
                        sch_year_data.to_excel(writer, sheet_name = f"{sch_str}_資料來源", index=False)
                    
        except Exception as e :
            error_message = traceback.format_exc()
            with open(os.path.join(os.path.dirname(os.getcwd()), "error", "圖型寫入錯誤"), "w", encoding='utf-8') as f:
                f.write(error_message)


            
        

